
```bash
conda deactivate
conda env remove --name networks

mamba create --prefix /DeepenData/.miniconda/envs/networks \
      scipy ray-default grpcio networkx pandas -c conda-forge -c anaconda --yes


conda activate networks

mamba install -C -S botocore boto3 --yes

pip install --upgrade setuptools
python -m pip install --upgrade pip

mamba install -c conda-forge apache-airflow 

conda env export --name networks > networks.yml
```



In [1]:


import warnings
warnings.filterwarnings("ignore")  #
import logging
import numpy as np
import networkx as nx
import pandas as pd
import ray

logging.basicConfig(level=logging.INFO)

def get_largest_connected_component(graph: nx.Graph) -> nx.Graph:
    """
    Find and return the largest connected component of a graph.

    Args:
        graph (nx.Graph): The input graph.

    Returns:
        nx.Graph: The largest connected subgraph.
    """
    assert isinstance(graph, nx.Graph), "Input must be a networkx Graph."
    
    try:
        largest_component = max(nx.connected_components(graph), key=len)
        largest_connected_subgraph = graph.subgraph(largest_component).copy()
        assert nx.is_connected(largest_connected_subgraph), "Output subgraph is not connected."
    except Exception as e:
        logging.error(f"Failed to compute largest connected component: {str(e)}")
        raise

    return largest_connected_subgraph



def calculate_quick_centralities(graph: nx.Graph) -> pd.DataFrame:
    """
    Compute some of the faster-to-calculate centralities and return them in a DataFrame.

    Args:
        graph (nx.Graph): The input graph.

    Returns:
        pd.DataFrame: DataFrame with the calculated centralities.
    """
    assert isinstance(graph, nx.Graph), "Input must be a networkx Graph."

    try:
        # Degree centrality
        degree_centrality = nx.degree_centrality(graph)

        # Eigenvector centrality
        eigenvector_centrality = nx.eigenvector_centrality(graph, max_iter=1000, tol=1e-05)

        # Closeness centrality
        closeness_centrality = nx.closeness_centrality(graph)

        # Information centrality
        info_centrality = nx.information_centrality(graph)

        centralities = {
            "degree_centrality": degree_centrality,
            "eigenvector_centrality": eigenvector_centrality,
            "closeness_centrality": closeness_centrality,
            "information_centrality": info_centrality,
        }

        df_centralities = pd.DataFrame(centralities)
    except Exception as e:
        logging.error(f"Failed to compute quick centralities: {str(e)}")
        raise

    return df_centralities

def calculate_centralities(graph: nx.Graph, use_quick_measurements: bool=False, alpha: float=0.005) -> pd.DataFrame:
    """
    Compute several centralities and return them in a DataFrame.

    Args:
        graph (nx.Graph): The input graph.
        use_quick_measurements (bool): Flag to use quick measurements. Default is False.
        alpha (float): The damping factor for Katz centrality and PageRank. Default is 0.005.

    Returns:
        pd.DataFrame: DataFrame with the calculated centralities.
    """
    assert isinstance(graph, nx.Graph), "Input must be a networkx Graph."
    assert isinstance(use_quick_measurements, bool), "use_quick_measurements must be a boolean."
    assert isinstance(alpha, (int, float)), "alpha must be a numeric type."

    try:
        if not use_quick_measurements:
            centralities = {
                "degree_centrality": nx.degree_centrality(graph),
                "harmonic_centrality": nx.harmonic_centrality(graph),
                "eigenvector_centrality": nx.eigenvector_centrality(graph, max_iter=1000, tol=1e-05),
                "betweenness_centrality": nx.betweenness_centrality(graph, normalized=True),
                "closeness_centrality": nx.closeness_centrality(graph, wf_improved=True),
                "load_centrality": nx.load_centrality(graph, normalized=True),
                "information_centrality": nx.information_centrality(graph),
                "katz_centrality": nx.katz_centrality(graph, alpha=alpha),
                "pagerank": nx.pagerank(graph, alpha=alpha),
            }
        else:
            centralities = calculate_quick_centralities(graph)
    except Exception as e:
        logging.error(f"Failed to compute centralities: {str(e)}")
        raise

    return pd.DataFrame(centralities)


def compute_alpha_for_graph(graph: nx.Graph) -> float:
    """
    Calculate the alpha parameter for Katz centrality and PageRank based on the graph's largest eigenvalue.

    Args:
        graph (nx.Graph): The input graph.

    Returns:
        float: The calculated alpha.
    """
    assert isinstance(graph, nx.Graph), "Input must be a networkx Graph."

    try:
        adjacency_matrix = nx.adjacency_matrix(graph).toarray()
        largest_eigenvalue = np.max(np.linalg.eigvals(adjacency_matrix))
        alpha = 0.9 * (1 / np.real(largest_eigenvalue))
    except Exception as e:
        logging.error(f"Failed to compute alpha: {str(e)}")
        raise

    return alpha



@ray.remote
def remove_node_and_calculate_centralities(graph: nx.Graph, node_to_remove, verbose: bool=False):
    """
    Remove a node from the graph, then calculate and return the centralities of the remaining nodes.

    Args:
        graph (nx.Graph): The input graph.
        node_to_remove: The node to be removed.
        verbose (bool): Flag to print verbose messages. Default is False.

    Returns:
        tuple: A tuple containing the removed node and a DataFrame of the new centralities.
    """
    assert isinstance(graph, nx.Graph), "Input must be a networkx Graph."
    assert isinstance(verbose, bool), "verbose must be a boolean."

    try:
        graph_copy = graph.copy()
        graph_copy.remove_node(node_to_remove)
        largest_connected_subgraph = get_largest_connected_component(graph_copy)
    except Exception as e:
        logging.error(f"Failed to compute largest connected component: {str(e)}")
        raise

    assert len(graph.nodes) != len(largest_connected_subgraph.nodes), "No node was removed."

    removed_nodes = set(graph.nodes) - set(largest_connected_subgraph.nodes)

    if verbose:
        logging.info(f"Nodes removed: {removed_nodes}")

    try:
        new_centralities = calculate_centralities(largest_connected_subgraph, use_quick_measurements=False, alpha=compute_alpha_for_graph(largest_connected_subgraph))
        new_centralities = new_centralities.reindex(list(graph.nodes))
        new_centralities.name = str(node_to_remove)
    except Exception as e:
        logging.error(f"Failed to calculate centralities: {str(e)}")
        raise

    # Check that removed nodes have NaN centralities
    for removed_node in removed_nodes:
        assert np.isnan(new_centralities.loc[removed_node, "eigenvector_centrality"]), "Removed node has non-NaN value."

    return node_to_remove, new_centralities



def remove_nodes_and_calculate_centralities(graph: nx.Graph, nodes_to_remove: list):
    """
    Remove a list of nodes from the graph and calculate the centralities after each removal.

    Args:
        graph (nx.Graph): The input graph.
        nodes_to_remove (list): List of nodes to be removed.

    Returns:
        list: List of tuples with removed node and new centralities.
    """
    # Checking input types
    assert isinstance(graph, nx.Graph), "Input must be a networkx Graph."
    assert isinstance(nodes_to_remove, list), "nodes_to_remove must be a list."

    # Check if all nodes to remove are in the graph, and create a list of nodes in the graph
    nodes_in_graph = [node for node in nodes_to_remove if node in graph.nodes()]

    # If any nodes_to_remove were not found in the graph, log a warning
    if len(nodes_to_remove) != len(nodes_in_graph):
        logging.warning(f"Not all nodes are in the graph ({len(nodes_in_graph)}/{len(nodes_to_remove)}).")

    try:
        # Here, we use Ray to execute the remove_node_and_calculate_centralities function in parallel for each node
        # The .remote() function call tells Ray to execute the function as a remote task
        # This line does not actually execute the tasks yet, but creates futures for them
        futures = [remove_node_and_calculate_centralities.remote(graph, node) for node in nodes_in_graph]

        # This line triggers the execution of the remote tasks and blocks until all tasks have completed
        # The results of the tasks are returned as a list in the same order as the futures
        result = ray.get(futures)
    except Exception as e:
        # If anything goes wrong during the execution of the tasks, log an error and re-raise the exception
        logging.error(f"Failed to remove nodes and calculate centralities: {str(e)}")
        raise

    # Return the result, which is a list of (removed_node, new_centralities) tuples
    return result

In [14]:
#import networkx as nx
 
graphml_path: str = "graph.graphml"
Gnx = nx.read_graphml(graphml_path)  #
Gnx = nx.Graph(Gnx)

G = Gnx.copy()

to_remove = list(Gnx.nodes)[60:70]


ray.init(ignore_reinit_error=True)


centralidades_perturbadas = remove_nodes_and_calculate_centralities(G, to_remove)

ray.shutdown()
baseline = calculate_centralities(G)

2023-05-31 10:46:26,297	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [13]:
print(
f'''
total nodes: {centralidades_perturbadas[0][1].shape[0]}
total centralites: {centralidades_perturbadas[0][1].shape[1]}
nodes removed: {len(centralidades_perturbadas)}
'''
)

10

total nodes: 1056
total centralites: 9
nodes removed: 10



In [19]:
# {'centralidades_perturbadas' : centralidades_perturbadas,
#  'baseline': baseline}
import pickle
 
 
with open('centralities.pickle', 'wb') as handle:
    pickle.dump({
        'centralidades_perturbadas' : centralidades_perturbadas,
        'baseline' : baseline
    }, handle, protocol=pickle.HIGHEST_PROTOCOL)

9

In [13]:
# Prepare a list to store all the DataFrames (baseline and perturbed centralities)
all_centralities = []
index_keys = []

# Add baseline centralities
all_centralities.append(baseline)
index_keys.append("baseline")

# Iterate over the perturbed centralities
for node in centralidades_perturbadas:
    # Get the centrality values and the name of the removed node
    perturbed_centralities = node[1]
    removed_node_name = node[0]

    # Add them to the list
    all_centralities.append(perturbed_centralities)
    index_keys.append(removed_node_name)

# Concatenate all DataFrames into one, with a multi-index
centralidades_df = pd.concat(
    all_centralities,  # DataFrames to concatenate
    axis=0,  # Concatenate row-wise
    keys=index_keys,  # Keys for the multi-index
    names=["removed_node", "metabolite"],  # Names for the levels of the multi-index
)


In [18]:
import pandas as pd
import numpy as np
from typing import List

class CentralityAnalyser:
    """Analyse centrality values for different removed nodes in a network."""

    def __init__(self, centralities_df: pd.DataFrame):
        """
        Initialize the centrality analyser with a dataframe of centrality values.
        
        Parameters
        ----------
        centralities_df : pd.DataFrame
            A dataframe containing the centrality values for different removed nodes.
        """
        self.centralities_df: pd.DataFrame = centralities_df.sort_index()
        self.baseline: pd.DataFrame = self.centralities_df.loc["baseline"]
    
    def calculate_log2_ratio_centrality(self, removed_node: str) -> pd.DataFrame:
        """
        Calculate the log2 ratio of centrality values relative to baseline for a given removed node.
        
        Parameters
        ----------
        removed_node : str
            The node that was removed.
            
        Returns
        -------
        pd.DataFrame
            A dataframe containing the log2 ratio of centrality values for the removed node.
        """
        assert all(self.baseline.index.values == self.centralities_df.loc[removed_node].index.values), "Index mismatch"

        centrality_ratio = np.log2(self.baseline / self.centralities_df.loc[removed_node])
        centrality_ratio["removed_node"] = removed_node

        return centrality_ratio.reset_index().set_index(["removed_node", "metabolite"])

    def calculate_centrality_variation(self) -> pd.DataFrame:
        """
        Calculate the centrality variation for all removed nodes.
        
        Returns
        -------
        pd.DataFrame
            A dataframe containing the centrality variation for all removed nodes.
        """
        all_removed_nodes: List[str] = self.centralities_df.index.get_level_values("removed_node").unique()

        centrality_variations = [self.calculate_log2_ratio_centrality(node) for node in all_removed_nodes]

        return pd.concat(centrality_variations)

# 
analyser = CentralityAnalyser(centralidades_df)
log2_ratio_df = analyser.calculate_centrality_variation()
log2_ratio_df

degree_centrality  harmonic_centrality  \
removed_node  metabolite                                                     
10FTHF5GLUtm  10FTHF5GLUtm                        NaN                  NaN   
              10FTHF5GLUtm_Neuron           -0.002738             0.001351   
              10FTHF6GLUtm                  -0.002738             0.004088   
              10FTHF6GLUtm_Neuron           -0.002738             0.001351   
              10FTHF7GLUtm                  -0.002738             0.002868   
...                                               ...                  ...   
sink_tyr-L(m) sink_tyr-L(m)_Neuron          -0.001368             0.000819   
              sink_vitd2                    -0.001368             0.001301   
              sink_vitd2_Neuron             -0.001368             0.000834   
              sink_vitd3                    -0.001368             0.001297   
              sink_vitd3_Neuron             -0.001368             0.000831   

                                    eigenvector_centrality  \
removed_node  metabolite                                     
10FTHF5GLUtm  10FTHF5GLUtm                             NaN   
              10FTHF5GLUtm_Neuron            -2.345901e-04   
              10FTHF6GLUtm                    6.412848e-05   
              10FTHF6GLUtm_Neuron            -2.345900e-04   
              10FTHF7GLUtm                   -2.193122e-06   
...                                                    ...   
sink_tyr-L(m) sink_tyr-L(m)_Neuron           -5.001942e-06   
              sink_vitd2                     -1.900514e-08   
              sink_vitd2_Neuron              -5.028246e-06   
              sink_vitd3                     -1.907357e-08   
              sink_vitd3_Neuron              -5.028596e-06   

                                    betweenness_centrality  \
removed_node  metabolite                                     
10FTHF5GLUtm  10FTHF5GLUtm                             NaN   
              10FTHF5GLUtm_Neuron                -0.002738   
              10FTHF6GLUtm                       -0.002738   
              10FTHF6GLUtm_Neuron                -0.002738   
              10FTHF7GLUtm                       -0.002738   
...                                                    ...   
sink_tyr-L(m) sink_tyr-L(m)_Neuron                     NaN   
              sink_vitd2                               NaN   
              sink_vitd2_Neuron                        NaN   
              sink_vitd3                               NaN   
              sink_vitd3_Neuron                        NaN   

                                    closeness_centrality  load_centrality  \
removed_node  metabolite                                                    
10FTHF5GLUtm  10FTHF5GLUtm                           NaN              NaN   
              10FTHF5GLUtm_Neuron              -0.001882        -0.002738   
              10FTHF6GLUtm                      0.001141        -0.002738   
              10FTHF6GLUtm_Neuron              -0.001883        -0.002738   
              10FTHF7GLUtm                      0.000504        -0.002738   
...                                                  ...              ...   
sink_tyr-L(m) sink_tyr-L(m)_Neuron             -0.000655              NaN   
              sink_vitd2                        0.000076              NaN   
              sink_vitd2_Neuron                -0.000638              NaN   
              sink_vitd3                        0.000075              NaN   
              sink_vitd3_Neuron                -0.000639              NaN   

                                    information_centrality  katz_centrality  \
removed_node  metabolite                                                      
10FTHF5GLUtm  10FTHF5GLUtm                             NaN              NaN   
              10FTHF5GLUtm_Neuron                -0.005308         1.167213   
              10FTHF6GLUtm                       -0.005273         1.140553   
          